This notebook allows you to regenerate `wikis.tsv` so that it incorporates any newly-created wikis or data changes.

In [1]:
import json
import re

import numpy as np
import pandas as pd
import requests
import wmfdata as wmf
from wmfdata.utils import get_dblist

## Basic information

In [2]:
# We start with all.dblist (instead of a sites table) because it excludes deleted wikis
wikis = get_dblist("all")
wikis = pd.DataFrame(wikis, columns=["database_code"])

# Left join to the sites table to get additional basic details
sites = wmf.mariadb.run("""
SELECT
    site_global_key AS database_code,
    CONCAT(TRIM(LEADING "." FROM REVERSE(site_domain))) AS domain_name,
    site_group AS database_group,
    site_language AS language_code
FROM sites
""", "metawiki")

wikis = (
    wikis
    .merge(sites, how="left", on="database_code", sort="True")
    .set_index("database_code")
)

/srv/home/nshahquinn-wmf/wmfdata-python/wmfdata/mariadb.py:142: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(


In [3]:
# Check for wikis with null data (e.g. newly created wikis not yet added to the sites table)
# If there are some, you can add the data manually in the following cell, but it may be more
# efficient to just wait a few days for the source data to be added.
wikis[wikis.isnull().any(axis=1)]

,domain_name,database_group,language_code
database_code,,,


In [4]:
# Add missing data. Remove manual additions once no longer needed
extra_wikis = pd.DataFrame([
    # Example: ("gcrwiki", "gcr.wikipedia.org", "wikipedia", "gcr"),
], columns=["database_code", "domain_name", "database_group", "language_code"]
).set_index("database_code")

wikis.update(extra_wikis, overwrite=False)

## Mobile domain names

In [5]:
# These are the mobile URL templates from the production wgMobileUrlTemplate config
# variable. As of Aug 2023, the variable is defined in
# https://gerrit.wikimedia.org/r/plugins/gitiles/operations/mediawiki-config
# /+/refs/heads/master/wmf-config/InitialiseSettings.php
#
# The templates here are kept in the same format and the same order as in the source
# variable, for ease of comparison.
default_template = "{h0}.m.{h1}.{h2}"
other_templates = {
    "mediawikiwiki": "m.{h1}.{h2}",
    "sourceswiki": "m.{h0}.{h1}",
    "wikidatawiki": "m.{h1}.{h2}",
    "wikifunctionswiki": "m.{h1}.{h2}",
    # "wikitech" is a database group that contains Wikitech (database code "labswiki") 
    # and Test Wikitech (database code "labtestwiki")
    "labswiki": "",
    "labtestwiki": ""
}

def derive_mobile_domain(row):
    split_domain = row["domain_name"].split(".")
    
    h_parts = {}
    # Assign the parts of the split domain to the names used in the templates ("h0", "h1", etc.)
    for i, p in enumerate(split_domain):
        h_parts[f"h{i}"] = p
    
    for db_code, template in other_templates.items():
        # The 'name' is the row's index
        if row.name == db_code:
            if template:
                return template.format(**h_parts)
            # An empty template means the desktop URL is used unmodified
            else:
                return ".".join(h_parts.values())
                
    return default_template.format(**h_parts)

wikis["mobile_domain_name"] = wikis.apply(derive_mobile_domain, axis=1)

# Language names

In [6]:
lang_urls = [
    "https://raw.githubusercontent.com/wikimedia/mediawiki-extensions-cldr/master/CldrNames/CldrNamesEn.php",
    "https://raw.githubusercontent.com/wikimedia/mediawiki-extensions-cldr/master/LocalNames/LocalNamesEn.php"
]

def get_lang_names(url):
    r = requests.get(url)
    m = re.search(r"languageNames = (\[[\s\S]+?\])", r.text)
    php_ln = m.group(1)
    
    json_ln = php_ln
    repl = [
        # Convert from PHP array format to JSON
        (" =>", ":"),
        ("\[", "{"),
        ("\]", "}"),
        # Trailing commas will cause problems
        (",\n}", "\n}"),
        # ...so will single quotes
        ("'", '"'),
        # Now, escaped double quotes (formerly escaped single quotes) can just be normal single quotes
        ('\\\\"', "'"),
        # ...and comments
        (r"/\*[\s\S]*?\*/", ""),
        (r"#(.*?)\n", ""),
        # One hack to deal with a single quote in a language name that was double quoted instead of 
        # escaped in the file
        ('O"odham', "O'odham"),
    ]
    for old, new in repl:
        json_ln = re.sub(old, new, json_ln)
    
    py_ln = json.loads(json_ln)
    return py_ln

langs = {}
for url in lang_urls:
    langs.update(get_lang_names(url))

wikis["language_name"] = wikis["language_code"].apply(langs.get)

In [7]:
# Check for wikis with null language names since some are not included in CLDR
null_lang_wikis = wikis[wikis["language_name"].isna()].copy()
set(null_lang_wikis["language_code"])

{'btm',
 'dga',
 'diq',
 'igl',
 'kus',
 'map-bms',
 'nah',
 'pih',
 'simple',
 'szy',
 'tay'}

In [8]:
# Add missing language names. Remove manual additions once no longer needed. 
extra_langs = {
    "btm": "Mandailing",
    "dga": "Dagaare",
    "diq": "Zazaki",
    "igl": "Igala",
    "kus": "Kusaal",
    "map-bms": "Banyumasan",
    "nah": "Nahuatl",
    "pih": "Norfuk-Pitkern",
    "simple": "Simple English",
    "szy": "Sakizaya",
    "tay": "Atayal"
}

null_lang_wikis["language_name"] = null_lang_wikis["language_code"].apply(extra_langs.get)
wikis.update(null_lang_wikis, overwrite=False)

## Status and access

In [9]:
closed = get_dblist("closed")
private = get_dblist("private")
fishbowl = get_dblist("fishbowl")
editability_private = private + fishbowl

wikis = (
    wikis.assign(
        status=lambda df: np.where(df.index.isin(closed), "closed", "open"),
        visibility=lambda df: np.where(df.index.isin(private), "private", "public"),
        editability=lambda df: np.where(df.index.isin(editability_private), "private", "public")
    )
)

## Site names

In [10]:
project_names = requests.get(
    "https://raw.githubusercontent.com/wikimedia/mediawiki-extensions-WikimediaMessages/master/" +
    "i18n/wikimediaprojectnames/en.json"
).text

replacements = [
    (r'"project-localized-name-(\w*)":', r'"\1":'),
    (r'"@metadata": \[.*\],', '')
]

for old, new in replacements:
    project_names = re.sub(old, new, project_names)

project_names = json.loads(project_names)
project_names = pd.DataFrame.from_dict(
    project_names, 
    orient="index", 
    columns=["english_name"]
).rename_axis("database_code")

wikis = wikis.merge(project_names, on="database_code", how="left")

In [11]:
# Check for wikis with null English names since some might not be included in WikimediaMessages' en.json
null_name_wikis = wikis[wikis["english_name"].isna()].copy()
null_name_wikis.index.tolist()

['arbcom_cswiki',
 'electcomwiki',
 'hiwikimedia',
 'projectcomwiki',
 'ptwikimedia',
 'romdwikimedia',
 'techconductwiki',
 'testcommonswiki',
 'wbwikimedia',
 'yuewiktionary']

How do we figure out what to name wikis for which we do not have names?

1. The type of the wiki might be identifiable through the database code. For example, all Wikipedia wikis end with `wiki`, and and all affiliate and user group wikis end with `wikimedia`. For Wikipedias, there's the [List of Wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias), and for other wikis there's the [Complete list of Wikimedia projects](https://meta.wikimedia.org/wiki/Special:MyLanguage/Complete_list_of_Wikimedia_projects).
2. Chapter, user group, and affiliate wikis tend to be named in either a way that ends with "Wikimedians User Group", or named after a country (e.g. "Wikimedia Portugal"). Visiting the wiki and using translation services can give an idea of what the name is, and some of these wikis are in English.
3. Committee wikis might have a database code that ends with `comwiki`. Again, finding the wikis URL and visiting it can give an idea of what the name is.
4. For most of these there are already existing wikis in the list, so querying it will give you an idea of what the naming scheme is.

In [12]:
# Add missing English names. Remove manual additions once no longer needed. 
extra_names = {
    "arbcom_cswiki" : "Czech Wikipedia Arbitration Committee",
    'electcomwiki' : "Wikimedia Foundation Elections Committee",
    'hiwikimedia' : "Hindi Wikimedians User Group",
    'projectcomwiki' : "Project Grants Committee",
    'ptwikimedia' : "Wikimedia Portugal",
    'romdwikimedia' : "Wikimedians of Romania and Moldova User Group",
    'techconductwiki' : "Code of Conduct Committee for Technical Spaces",
    'testcommonswiki' : "Commons Test Wiki",
    'wbwikimedia' : "West Bengal Wikimedians User Group ",
    'yuewiktionary' : "Cantonese Wiktionary"
}

null_name_wikis["english_name"] = null_name_wikis.index.map(extra_names.get)
wikis.update(null_name_wikis, overwrite=False)

## Write to file

In [13]:
wikis.to_csv("wikis.tsv", sep="\t")